In [8]:
import pandas as pd
START_DATA = pd.read_csv('data/merged_dataset.csv', nrows = 1000)
nan_cols = START_DATA.isna().any()

cols_with_nan = START_DATA.columns[nan_cols]

df = START_DATA.copy()

df = df.drop(columns=cols_with_nan)

df = df.select_dtypes(exclude=['object'])

float64_cols = list(df.select_dtypes(include=['float64']))
int64_cols = list(df.select_dtypes(include=['int64']))


cols_to_read = df.columns

In [ ]:
cols_to_read_final = list(filter(lambda x: len(x) < 14, cols_to_read))

In [5]:
len(cols_to_read_final)

12407

In [7]:
float64_cols_dict = {}
for x in float64_cols:
    float64_cols_dict[x] = 'float32'
float64_cols_dict
int64_cols_dict = {}
for x in int64_cols:
    int64_cols_dict[x] = 'uint8'
dtype_to_read = {**float64_cols_dict, **int64_cols_dict}

In [8]:
FULL_DATA = pd.read_csv('data/merged_dataset.csv', usecols=cols_to_read_final, dtype=dtype_to_read, nrows=100)

In [9]:
FULL_DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 12407 entries, day_tempmax to zyuganov
dtypes: float32(15), uint8(12392)
memory usage: 1.2 MB


In [10]:
FULL_DATA.head()

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_uvindex,day_moonphase,hour_temp,hour_humidity,...,zuhr,zurab,zusko,zvanivka,zvezda,zvinchuk,zyabrovka,zybyn,zymohiria,zyuganov
0,1.6,-4.1,-1.6,-5.3,76.800003,0.1,3.0,0.94,-2.2,76.339996,...,0,0,0,0,0,0,0,0,0,0
1,1.6,-4.1,-1.6,-5.3,76.800003,0.1,3.0,0.94,-2.7,76.839996,...,0,0,0,0,0,0,0,0,0,0
2,1.6,-4.1,-1.6,-5.3,76.800003,0.1,3.0,0.94,-3.7,80.230003,...,0,0,0,0,0,0,0,0,0,0
3,1.6,-4.1,-1.6,-5.3,76.800003,0.1,3.0,0.94,-3.7,80.279999,...,0,0,0,0,0,0,0,0,0,0
4,1.6,-4.1,-1.6,-5.3,76.800003,0.1,3.0,0.94,-3.9,80.870003,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.model_selection import TimeSeriesSplit, train_test_split

#df = pd.read_csv('data/final_data.csv')
df = FULL_DATA

In [12]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
(df.dtypes == object).any()

False

In [14]:
X = df[df.columns.difference(['alarm_info'])]
y = df['alarm_info']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)


tscv = TimeSeriesSplit()


In [22]:
X_train

,01,05000600,08000830,10,100,1000,10000,100000,1000000,100000rubl,...,zuhr,zurab,zusko,zvanivka,zvezda,zvinchuk,zyabrovka,zybyn,zymohiria,zyuganov
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
del START_DATA
del FULL_DATA

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import TimeSeriesSplit, train_test_split

**Logistic Regression without Cross Validation**

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=50000)

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(logreg.score(X_test, y_test))

In [ ]:
import joblib

joblib.dump(logreg, 'trained_models/LogisticRegression')

**Logistic Regression with Cross Validation**

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
logregCV = LogisticRegressionCV(cv=tscv, max_iter = 100000)

logregCV.fit(X_train, y_train)

y_pred = logregCV.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(logregCV.score(X_test, y_test))

**Support Vector Machine**

In [ ]:
from sklearn import svm

#max_iter should probably be changed when training on actual data
SVM = svm.SVC(max_iter=50000)

SVM.fit(X_train, y_train)

y_pred = SVM.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(SVM.score(X_test, y_test))

**Stochastic Gradient Descent**

In [ ]:
from sklearn.linear_model import SGDClassifier

#max_iter should probably be changed when training on actual data
SGD = SGDClassifier(loss="modified_huber", penalty="l2", max_iter=50000) 

SGD.fit(X_train, y_train)

y_pred = SGD.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(SGD.score(X_test, y_test))

**Random Forest Classifier with max depth 5**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC5 = RandomForestClassifier(max_depth = 5, random_state=0)

RFC5.fit(X_train, y_train)

y_pred = RFC5.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(RFC5.score(X_test, y_test))

In [ ]:
joblib.dump(RFC5, 'trained_models/RandomForestClassifier5')

**Random Forest Classifier with max depth 20**

In [ ]:
RFC20 = RandomForestClassifier(max_depth = 20, random_state=0)

RFC20.fit(X_train, y_train)

y_pred = RFC20.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(RFC20.score(X_test, y_test))

In [ ]:
joblib.dump(RFC20, 'trained_models/RandomForestClassifier20')

**Random Forest Classifier with max depth 50**

In [ ]:
RFC50 = RandomForestClassifier(max_depth = 50, random_state=0)

RFC50.fit(X_train, y_train)

y_pred = RFC50.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(RFC50.score(X_test, y_test))

In [ ]:
joblib.dump(RFC50, 'trained_models/RandomForestClassifier50')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC100 = RandomForestClassifier(max_depth = 100, random_state=0)

RFC100.fit(X_train, y_train)

y_pred = RFC100.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(RFC100.score(X_test, y_test))

In [ ]:
joblib.dump(RFC100, 'trained_models/RandomForestClassifier100')

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

QDA = QuadraticDiscriminantAnalysis()

QDA.fit(X_train, y_train)

y_pred = QDA.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(QDA.score(X_test, y_test))

**k-NN with k = 5**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(knn.score(X_test, y_test))\

In [ ]:
import joblib
joblib.dump(knn, 'trained_models/5NearestNeighbours')

**k-NN with k = 20**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=20)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(knn.score(X_test, y_test))

In [ ]:
joblib.dump(knn, 'trained_models/20NearestNeighbours') 

**k-NN with k = 50**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=50)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(knn.score(X_test, y_test))

In [ ]:
joblib.dump(knn, 'trained_models/50NearestNeighbours') 